Notebook per ajuntar i guardar nous datasets

In [6]:
import pickle
import pandas as pd



hospital = r"data\timeseries_pickles\additional_features\weekly_hospital_data.pkl"
vaccines = r"data\timeseries_pickles\additional_features\weekly_vaccine_feats.pkl"
training = r"data\training_data\training_datav2_0-weekly.pkl"


with open(hospital, 'rb') as file:
    hospital_df = pd.DataFrame(pickle.load(file))
    hospital_df.rename(columns={'entity': 'location'}, inplace=True)
    hospital_df.drop(columns=['iso_code'], inplace=True)
with open(vaccines, 'rb') as file:
    vaccines_df = pd.DataFrame(pickle.load(file))
    
with open(training, 'rb') as file:
    training_df = pd.DataFrame(pickle.load(file))

training_df.location.unique()

array(['Algeria', 'Argentina', 'Australia', 'Austria', 'Belgium',
       'Bolivia', 'Bulgaria', 'Canada', 'Chile', 'Croatia', 'Cyprus',
       'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece',
       'Hungary', 'Iceland', 'Ireland', 'Israel', 'Italy', 'Japan',
       'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Malaysia',
       'Malta', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania',
       'Serbia', 'Singapore', 'Slovenia', 'South Africa', 'South Korea',
       'Spain', 'Sweden', 'Switzerland', 'United Kingdom',
       'United States'], dtype=object)

In [7]:
# Ensure 'date' is in datetime format
# print(training_df.location.unique())
training_df['date'] = pd.to_datetime(training_df['date'])
vaccines_df['date'] = pd.to_datetime(vaccines_df['date'])
hospital_df['date'] = pd.to_datetime(hospital_df['date'])

# Strip whitespace and standardize 'location' to lowercase
training_df['location'] = training_df['location'].str.strip().str.lower()
vaccines_df['location'] = vaccines_df['location'].str.strip().str.lower()
hospital_df['location'] = hospital_df['location'].str.strip().str.lower()

# Drop rows with missing 'date' or 'location' values
# print("NaN values in training_df:")
# print(training_df.isna().sum())

# print("\nNaN values in vaccines_df:")
# print(vaccines_df.isna().sum())

# print("\nNaN values in hospital_df:")
# print(hospital_df.isna().sum())
training_df = training_df.dropna(subset=['date', 'location'])
vaccines_df = vaccines_df.dropna(subset=['date', 'location'])
hospital_df = hospital_df.dropna(subset=['date', 'location'])

# Delete locations which are not available in both datasets
locations_in_training = set(training_df['location'].unique())
locations_in_vaccines = set(vaccines_df['location'].unique())
locations_only_in_training = locations_in_training - locations_in_vaccines
locations_only_in_vaccines = locations_in_vaccines - locations_in_training

training_df = training_df[~training_df['location'].isin(locations_only_in_training)]
vaccines_df = vaccines_df[~vaccines_df['location'].isin(locations_only_in_vaccines)]


# Merge the datasets on 'date' and 'location'
merged_df = training_df.merge(vaccines_df, on=['date', 'location'], how='left').fillna(0)
                       #.merge(hospital_df, on=['date', 'location'], how='inner')

# Drop rows with any remaining NaN values after the merge
merged_df = merged_df.dropna()

# Save the merged dataset
merged_df.to_pickle(r"data\training_data\training_added_feats_no_hospital.pkl")

In [8]:
vaccines_df.shape

(2484, 4)

In [5]:
merged_df

,location,date,weekly_cases,C,E,G,S,biweekly_cases,biweekly_deaths,new_cases,new_deaths,total_cases,total_deaths,weekly_deaths,Weekly new ICU admissions,Weekly new ICU admissions per million,Weekly new hospital admissions,Weekly new hospital admissions per million
0,algeria,2020-01-12,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,0.0,0.0
1,algeria,2020-01-19,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,0.0,0.0
2,algeria,2020-01-26,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,0.0,0.0
3,algeria,2020-02-02,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,0.0,0.0
4,algeria,2020-02-09,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5923,united states,2022-12-04,300171.0,42.581429,0.0,37.260000,28.740000,579100.000000,4758.285714,43228.428571,251.857143,9.734388e+07,1.069745e+06,2496.714286,0.0,0.0,0.0,0.0
5924,united states,2022-12-11,302599.0,42.580000,0.0,37.260000,28.730000,625920.285714,4441.428571,66031.857143,433.571429,9.766928e+07,1.071690e+06,1944.714286,0.0,0.0,0.0,0.0
5925,united states,2022-12-18,462223.0,42.574286,0.0,37.254286,28.725714,785309.857143,4925.000000,63716.285714,378.857143,9.812919e+07,1.074670e+06,2980.285714,0.0,0.0,0.0,0.0
5926,united states,2022-12-25,446014.0,42.570000,0.0,37.250000,28.720000,914777.857143,5686.714286,72572.714286,433.285714,9.858406e+07,1.077376e+06,2706.428571,0.0,0.0,0.0,0.0


In [12]:
hospital_data_path = r"data\pickles\hospital_data.pkl"

with open(hospital_data_path, 'rb') as file:
    hospital_data_df = pd.DataFrame(pickle.load(file))

# Display the first few rows of the loaded dataframe
hospital_data_df.entity.nunique()

50

In [9]:
['C',
 'E',
 'G',
 'S',
 'daily_vaccinations_per_million',
 'Weekly new ICU admissions per million',
 'Weekly new hospital admissions per million']

['C',
 'E',
 'G',
 'S',
 'daily_vaccinations_per_million',
 'Weekly new ICU admissions per million',
 'Weekly new hospital admissions per million']

In [10]:
print(merged_df.location.nunique())
print(vaccines_df.location.nunique())
print(hospital_df.location.nunique())

38
38
17


# policies desgregades

muntar el dataset amb new_cases + les policies individuals desgregades

In [2]:
import pandas as pd

In [6]:
policy = pd.read_pickle(r'data\all_policies\policies_individual_final.pkl')
training = pd.read_pickle(r'data\training_data\training_datav2_0-weekly.pkl')

merged_policy_df = training.merge(policy, on=['location', 'date'], how='inner')
merged_policy_df = merged_policy_df.drop(columns= ['C', 'E', 'G', 'S'])
print(f"Original training shape: {training.shape}")
print(f"Original policy shape: {policy.shape}")
print(f"Merged shape: {merged_policy_df.shape}")

Original training shape: (6864, 14)
Original policy shape: (6864, 18)
Merged shape: (6864, 26)


In [8]:
merged_policy_df.columns

Index(['location', 'date', 'weekly_cases', 'biweekly_cases', 'biweekly_deaths',
       'new_cases', 'new_deaths', 'total_cases', 'total_deaths',
       'weekly_deaths', 'c1m', 'c2m', 'c3m', 'c4m', 'c5m', 'c6m', 'c7m',
       'c8ev', 'e1', 'e2', 'h1', 'h2', 'h3', 'h6m', 'h7', 'h8m'],
      dtype='object')

# hospital

In [54]:
import pandas as pd
def get_pd_from_pkl(path: str) -> pd.DataFrame:
    df = pd.read_pickle(path)
    return df

In [55]:
df_hospital = get_pd_from_pkl(path=r'data\pickles\hospital_data.pkl')
df_og = get_pd_from_pkl(path=r'data\training_data\training_datav2_0-weekly.pkl')

df_hospital['date'] = pd.to_datetime(df_hospital['date'])
countries = df_og['location'].unique()
dates = df_og['date'].unique()

In [56]:
temp = df_hospital[(df_hospital['entity'].isin(countries))]
temp = temp.pivot(index=['entity', 'iso_code', 'date'], columns='indicator', values='value').reset_index()
temp
# Calculate global NaN ratios for the selected columns
cols_to_check = [
    'Daily ICU occupancy per million',
    'Daily hospital occupancy per million',
]
global_nan_ratios = temp[cols_to_check].isna().mean()
print(global_nan_ratios)


indicator
Daily ICU occupancy per million         0.172225
Daily hospital occupancy per million    0.156181
dtype: float64


In [57]:
df_mod = df_hospital.loc[df_hospital['entity'].isin(countries) & df_hospital['date'].isin(dates)]
# Pivot the DataFrame
df_mod = df_mod.pivot(index=['entity', 'iso_code', 'date'], columns='indicator', values='value').reset_index()

In [58]:
def resample_country_data(group):
    # Separate numerical and categorical columns
    numerical_cols = group.select_dtypes(include=['number']).columns
    categorical_cols = ['entity', 'iso_code'] # Add your categorical columns here

    # Resample numerical columns (average for the week)
    weekly_numerical = group[numerical_cols].resample('W-SUN').mean()

    # Resample categorical columns (e.g., take the first value of the week)
    weekly_categorical = group[categorical_cols].resample('W-SUN').first()

    # Combine numerical and categorical data
    weekly_data = pd.concat([weekly_numerical, weekly_categorical], axis=1)
    return weekly_data

temp = temp.set_index('date')
temp = temp.groupby('entity', group_keys=False).apply(resample_country_data)
temp = temp.reset_index()
temp = temp.set_index('date')
temp['Daily hospital occupancy per million'] = temp['Daily hospital occupancy per million'].interpolate(method='time')
nan_rate = temp['Daily hospital occupancy per million'].isna().mean()
print(f"NaN rate for 'Daily hospital occupancy per million': {nan_rate:.4f}")

NaN rate for 'Daily hospital occupancy per million': 0.0310


C:\Users\gsamp\AppData\Local\Temp\ipykernel_5960\1135051526.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  temp = temp.groupby('entity', group_keys=False).apply(resample_country_data)


In [59]:
temp.head()

indicator,Daily ICU occupancy,Daily ICU occupancy per million,Daily hospital occupancy,Daily hospital occupancy per million,Weekly new ICU admissions,Weekly new ICU admissions per million,Weekly new hospital admissions,Weekly new hospital admissions per million,entity,iso_code
date,,,,,,,,,,
2020-07-19,64.500000,1.4365,NaN,NaN,NaN,NaN,NaN,NaN,Algeria,DZA
2020-07-26,60.666667,1.3510,NaN,NaN,NaN,NaN,NaN,NaN,Algeria,DZA
2020-08-02,60.250000,1.3420,NaN,NaN,NaN,NaN,NaN,NaN,Algeria,DZA
2020-08-09,57.250000,1.2750,NaN,NaN,NaN,NaN,NaN,NaN,Algeria,DZA
2020-08-16,44.500000,0.9910,NaN,NaN,NaN,NaN,NaN,NaN,Algeria,DZA


In [64]:
final_df = temp[['Daily hospital occupancy per million', 'entity']].reset_index().fillna(0)
final_df = final_df.loc[(final_df['date'].isin(dates)) & (final_df['entity'].isin(countries))]
print(final_df['entity'].value_counts())
final_df.to_pickle(r'data\timeseries_pickles\additional_features\new_weekly_hospital_data.pkl')

entity
Greece            152
Estonia           152
Latvia            152
Luxembourg        150
Slovenia          150
Italy             149
Ireland           149
Germany           149
Denmark           149
Switzerland       149
Serbia            148
Belgium           148
Liechtenstein     148
Israel            148
Malta             148
Cyprus            147
Sweden            147
France            146
Malaysia          145
South Korea       145
Australia         144
United Kingdom    144
Canada            144
Austria           144
Chile             144
Norway            143
Finland           142
Bulgaria          141
Portugal          141
Japan             140
United States     129
Spain             127
Algeria           127
Poland            126
South Africa      126
Croatia           123
Lithuania         114
Netherlands       105
Romania           104
Argentina          95
Iceland            79
Bolivia            76
Name: count, dtype: int64


In [ ]:
final